THIS IS THE NOTEBEOOK WHERE WE ARE GOING TO FIRST INITIALIZE A MODEL AND THEN THEN EMBEDD THE DATA INTO THE RAG PIPELINE


In [1]:
import pandas as pd
from langchain_core.documents import Document

# Load the FIFA matches data
df = pd.read_csv("data/Fifa_world_cup_matches.csv")  # adjust path if needed

# Quick peek
print(df.head())
print(df.columns)


           team1         team2 possession team1 possession team2  \
0          QATAR       ECUADOR              42%              50%   
1        ENGLAND          IRAN              72%              19%   
2        SENEGAL   NETHERLANDS              44%              45%   
3  UNITED STATES         WALES              51%              39%   
4      ARGENTINA  SAUDI ARABIA              64%              24%   

  possession in contest  number of goals team1  number of goals team2  \
0                    8%                      0                      2   
1                    9%                      6                      2   
2                   11%                      0                      2   
3                   10%                      1                      1   
4                   12%                      1                      2   

          date     hour category  ...  penalties scored team1  \
0  20 NOV 2022  17 : 00  Group A  ...                       0   
1  21 NOV 2022  14 : 0

converting each row of the data into text to embedded into the vector Store

In [ ]:
df.columns


In [ ]:
df.columns

In [2]:
def row_to_text(row):
    # Adjust column names to your actual CSV headers
    return (
       
        f"FIFA World Cup 2022 Match {row['team1']} vs {row['team2']}. "
        f"Score: {row['number of goals team1']} - {row['number of goals team2']}. "
        f"Date: {row['date']}. "
        f"Category:{row['category']} ,"
        f"Possesion of {row['team1']} is {row['possession team1']} and {row['team2']} is {row['possession team2']} "
        f"{row['team1']} had {row['on target attempts team1']} On Target Attempts and {row['team2']} had  {row['on target attempts team2']} attempts"
         f"{row['team1']} had {row['assists team1']} assists and {row['team2']} had {row['assists team2']} assists"
    )

docs= []
for _, row in df.iterrows():
    text = row_to_text(row)
    metadata = {
        "team1": row["team1"],
        "team2": row["team2"],
        "category":row['category']
    }
    docs.append(Document(page_content=text, metadata=metadata))

len(docs), docs[0]


(64,
 Document(metadata={'team1': 'QATAR', 'team2': 'ECUADOR', 'category': 'Group A'}, page_content='FIFA World Cup 2022 Match QATAR vs ECUADOR. Score: 0 - 2. Date: 20 NOV 2022. Category:Group A ,Possesion of QATAR is 42% and ECUADOR is 50% QATAR had 0 On Target Attempts and ECUADOR had  3 attemptsQATAR had 0 assists and ECUADOR had 1 assists'))

In [ ]:
print(docs)

INTIATING THE EMBEDDING MODEL

In [3]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)



In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from pathlib import Path
from langchain_core.documents import Document  

def load_and_chunk_all_text_files(data_dir="data", chunk_size=500, chunk_overlap=100):
    """
    Load all .txt files from data directory and chunk them recursively
    
    Args:
        data_dir: Path to data directory
        chunk_size: Size of each chunk
        chunk_overlap: Overlap between chunks
    
    Returns:
        List of Document objects with chunked content
    """
    
    # Initialize text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    docs = []
    data_path = Path(data_dir)
    
    # Recursively find all .txt files
    for txt_file in data_path.rglob("*.txt"):
        try:
            with open(txt_file, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # Split the text into chunks
            chunks = text_splitter.split_text(content)
            
            # Create documents from chunks
            for i, chunk in enumerate(chunks):
                doc = Document(
                    page_content=chunk,
                    metadata={
                        "source": str(txt_file),
                        "file_name": txt_file.stem,
                        "chunk_index": i,
                        "total_chunks": len(chunks)
                    }
                )
                docs.append(doc)
                
            print(f"✅ Loaded {txt_file.name}: {len(chunks)} chunks")
        
        except Exception as e:
            print(f"❌ Error loading {txt_file.name}: {e}")
    
    print(f"\n📊 Total documents created: {len(docs)}")
    return docs

# Usage in your notebook:
# text_docs = load_and_chunk_all_text_files("data")
# Then add these to your vector store along with CSV docs

In [7]:
from langchain_community.vectorstores import FAISS
csv_docs = docs  # your existing CSV documents

# Load and chunk all txt files
text_docs = load_and_chunk_all_text_files("data")

# Combine both
all_docs = csv_docs + text_docs

# Create vector store with all documents
vector_store = FAISS.from_documents(all_docs, embedding=embeddings)

✅ Loaded argentina_france_F.txt: 18 chunks
✅ Loaded 03_France_vs_Poland.txt: 2 chunks
✅ Loaded england_france_Q.txt: 16 chunks
✅ Loaded portugal_morroco_Q.txt: 28 chunks
✅ Loaded croatia_brazil_Q.txt: 18 chunks
✅ Loaded 06_Brazil_vs_South_Korea.txt: 1 chunks
✅ Loaded 07_Morocco_vs_Spain.txt: 1 chunks
✅ Loaded 05_Japan_vs_Croatia.txt: 2 chunks
✅ Loaded croatia_morroco_TH.txt: 0 chunks
✅ Loaded 08_Portugal_vs_Switzerland.txt: 1 chunks
✅ Loaded 04_England_vs_Senegal.txt: 2 chunks
✅ Loaded argentina_netherlands_Q.txt: 0 chunks
✅ Loaded 01_Netherlands_vs_USA.txt: 3 chunks
✅ Loaded argentina_croatia_SF.txt: 15 chunks
✅ Loaded 02_Argentina_vs_Australia.txt: 3 chunks
✅ Loaded franve_morroco_SF.txt: 14 chunks

📊 Total documents created: 124


TEXT SPLITTER FOR THE RECAPS DATA


Vector Storing

Now that the vector embeddings are completed we will now be doing a test for similarity search so that we can see what is actually going on here now using similiratiy_score_threshhold

similarity = “just give me closest stuff”

similarity_score_threshold = “closest stuff but drop trash”

mmr = “closest but also diverse (good for longer answers)”

In [8]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 10,                 # consider up to 10
        "score_threshold": 0.9   # filter out low similarity
    }
)



In [9]:
retriever_s = vector_store.as_retriever(
    search_type="similarity",               # or "similarity_score_threshold", "mmr"
    search_kwargs={"k": 5}                  # top 5 matches
)


In [10]:
prompt = "Tell me about the match between Portugal and Morocco"  # ⬅️ Add this

results = retriever_s.invoke(prompt)   # ⬅️ this replaces get_relevant_documents

for doc in results:
    print(doc.page_content)
    print(doc.metadata)
    print("-" * 40)

FIFA World Cup 2022 Match MOROCCO vs PORTUGAL. Score: 1 - 0. Date: 10 DEC 2022. Category:Quarter-final ,Possesion of MOROCCO is 22% and PORTUGAL is 65% MOROCCO had 3 On Target Attempts and PORTUGAL had  3 attemptsMOROCCO had 1 assists and PORTUGAL had 0 assists
{'team1': 'MOROCCO', 'team2': 'PORTUGAL', 'category': 'Quarter-final'}
----------------------------------------
FIFA World Cup 2022 Match MOROCCO vs SPAIN. Score: 0 - 0. Date: 06 DEC 2022. Category:Round of 16 ,Possesion of MOROCCO is 22% and SPAIN is 68% MOROCCO had 3 On Target Attempts and SPAIN had  1 attemptsMOROCCO had 0 assists and SPAIN had 0 assists
{'team1': 'MOROCCO', 'team2': 'SPAIN', 'category': 'Round of 16'}
----------------------------------------
. Morocco had less possession at every stage of the match, but scored the only goal in the 42nd minute, during the 15-minute interval where they had their highest possession percentage (43% possession between minute 30-45+). Portugal had 81% of their possessions reach th

Now Setting Up The Actualy LLM and then seeing what can be done

In [11]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from langchain.tools import tool,ToolRuntime

In [12]:
llm = ChatOllama(
    model="llama3.2:3b",  # or any local ollama model name you’ve pulled
    temperature=0.2,
)

checkpointer = InMemorySaver()

In [13]:

from langchain_core.tools import create_retriever_tool

retriever_tool = create_retriever_tool(retriever_s,name='retriever_data',description='Usign this tool we will retrive the data from the vector store')



In [14]:
@tool
def retrieve_data(query: str) -> str:
    """Retrieve World Cup match data from the vector store"""
    results = retriever_s.invoke(query)
    return "\n\n".join([doc.page_content for doc in results])

checkpointer = InMemorySaver()

agent = create_agent(
    model=llm,
    tools=[retrieve_data],
    checkpointer=checkpointer,
    system_prompt="""You are a FIFA Football Retrieval Assistant.
Your job is to answer questions only using the information retrieved from the provided knowledge base or documents.

Rules:

Always perform a retrieval step before answering.
If nothing relevant is found, say:

“No data available in the current database.”

Never invent facts.
No guessing, no assumptions beyond retrieved content.

Keep answers concise, factual, and match football terminology.

If the user asks for stats, show them in a neat bullet list or table.

If multiple retrieved documents conflict, mention the conflict and do NOT pick a side.

Do not summarize entire documents; answer only the question asked.

Style:

Friendly but direct, like a football commentator who knows their stats.

If a result involves teams or players, always mention season, competition, and score if available.

When giving match recaps, include:

final score

goal scorers

major events (cards, penalties, substitutions)

Clarifications:

If the user asks something vague (“How was Madrids season?”), request a specific season/year before proceeding.

If the query is unrelated to football, politely refuse."""
)

In [15]:
config={"configurable": {"thread_id": "9"}}

result=agent.invoke({
    "messages":[{"role":"user","content":"Hey I am Ananda Prem How are you doing"}]
},config)

resulted=agent.invoke({
    "messages":[{"role":"user","content":"Tell Me about Portugal Vs Morroc's match in the FIFA 2022 world cup"}]
},config)

result_2=agent.invoke({
    "messages":[{"role":"user","content":"What would you say about the key moment where their fate in the worldcup was decided"}]
},config)



In [16]:
print("Result 1:")
print(resulted["messages"][-1].content)
print("\nResult 2:")
print(result_2["messages"][-1].content)

print("\nResult 3:")

print(result["messages"][-1].content)

Result 1:
Here's a summary of the match:

**Match Details:**
Portugal vs Morocco, FIFA World Cup 2022
Date: December 10, 2022
Score: 1-0 (Morocco won)

**Goal Scorer:** Youssef En-Nesyri (42')

**Possession:** Morocco (22%) and Portugal (65%)

**On Target Attempts:** Morocco (3) and Portugal (3)
**Assists:** Morocco (1) and Portugal (0)

This was a crucial match in the quarter-finals, where Morocco pulled off an upset victory against the 9th-ranked Portugal team.

Result 2:
The key moment where Morocco's fate in the World Cup was decided was when Youssef En-Nesyri scored the only goal of the match in the 42nd minute. This goal gave Morocco a crucial advantage in the quarter-final match against Portugal, ultimately leading to their 1-0 victory and advancement to the semi-finals.

It's worth noting that this moment marked a turning point in the match, as it shifted the momentum in favor of Morocco and put pressure on Portugal to respond. The goal was a result of a well-crafted team effor

In [ ]:
print("Result 3 (Full):")
print(result_3["messages"][-1].content)


In [ ]:
result["messages"][-1].pretty_print()
result_2["messages"][-1].pretty_print()
result_3["messages"][-1].pretty_print()


In [17]:
config={"configurable": {"thread_id": "1"}}

agent.invoke({"messages": "hi, my name is Nathan"}, config)
resp=agent.invoke({"messages": "Tell me about Portugals Match Against Morroco"}, config)
resp=agent.invoke({"messages": "What Would You tell About the possession did portugal deserve to win based off the possesion they maintained throught the match"}, config)
resp["messages"][-1].pretty_print()
resp=agent.invoke({"messages":"how many attempts did portugal have in this match compared to morroco?"},config)
resp["messages"][-1].pretty_print()


================================== Ai Message ==================================

Based on the data, it appears that Portugal did not deserve to win the match based on possession alone. Although they had more possession overall (65% vs 22%), Morocco was able to capitalize on their opportunities and score the only goal of the game.

In fact, an analysis of ball possession during a previous tournament found that successful teams tend to have greater possession in the middle offensive zone, but this was not the case for Portugal against Morocco. Additionally, Morocco's ability to maintain possession and create scoring chances suggests that they may have been the more deserving team to win the match.

It's worth noting that possession is just one metric, and other factors such as shot quality, passing accuracy, and defensive solidity can also play a role in determining which team deserves to win. However, based on the data available, it appears that Morocco's performance was more impressiv

In [18]:
config={"configurable": {"thread_id": "1"}}
resp=agent.invoke({"messages":"What About Total Attempts On and Off Target"},config)
resp["messages"][-1].pretty_print()

================================== Ai Message ==================================

According to the data, here are the total attempts on and off target for both teams:

* Morocco:
	+ Total attempts: 4
	+ On-target attempts: 1 (resulting in a goal)
	+ Off-target attempts: 3
* Portugal:
	+ Total attempts: 3
	+ On-target attempts: 2
	+ Off-target attempts: 1

This suggests that Morocco had more total attempts, but only one of them resulted in a goal. Portugal, on the other hand, had two on-target attempts, but neither of them resulted in a goal.


Creating a Function to Get Input from a User instead Of hardCoding it

its now inside main_func.py from where we are going to call it

In [ ]:
from main_func import ask_fifa_bot


In [ ]:
# Test with new thread and different question
print(ask_fifa_bot("Tell me about Brazil vs South Korea match", "new_user_123"))
print("\n" + "="*60 + "\n")
print(ask_fifa_bot("What were the possession stats for Japan vs Croatia?", "new_user_456"))

In [ ]:
print(ask_fifa_bot("Tell me about Brazil vs South Korea match", "new_user_023"))

HElllo